In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["PINECODE_API_KEY"]=os.getenv("PINECODE_API_KEY")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"]=os.getenv("HUGGINGFACE_API_KEY")


In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/miniconda3/envs/agentic_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embed=embeddings.embed_query("What's our Q1 revenue?")
len(embed)

768

In [8]:
from pinecone import Pinecone
pinecone_api_key=os.getenv("PINECODE_API_KEY")
pinecone_api_key

'pcsk_6ozawh_RgpBbfMY8omfbkyr97tGmyYRmukGiCQKGWNsYLeoZ7iR1ETH1rCskngPcXarSpS'

In [10]:
pc=Pinecone(api_key=pinecone_api_key)

In [ ]:
from pinecone import ServerlessSpec

index_name = "agentic-2"  # change if desired
#pc.delete_index(name=index_name)
if not pc.has_index(index_name):
    print("creating index")
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

creating index


In [14]:
from langchain_pinecone import PineconeVectorStore

vector_store=PineconeVectorStore(
    index=index,
    embedding=embeddings
)

In [15]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},#additional info
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

In [16]:
    documents = [
        document_1,
        document_2,
        document_3,
        document_4,
        document_5,
        document_6,
        document_7,
        document_8,
        document_9,
        document_10,
    ]

In [17]:
documents

[Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
 Document(metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic application

In [18]:
vector_store.add_documents(documents)

['6fe69a71-74e6-4c18-a04f-84173c3aefe6',
 'b219dd34-c146-4dd7-9af7-39a6365d939e',
 'f28d1be9-5f8c-485a-bc0a-fd29982a48f2',
 'f28917fb-c36e-4e9c-b06e-81f4200f0b88',
 '35eeeff2-dfb9-46aa-a7aa-c6da87ddee13',
 'c40baf51-2fcd-4fbc-822a-a61796907c6f',
 'fcb63eb4-56bf-4c12-8925-2c56e09bd723',
 '025ad75a-4414-4e2a-88f5-ee2a5367197d',
 '76d6e1d2-8e46-469c-bdda-422c3c600b57',
 'b32021c8-9c46-4fd1-9c6c-b77939a975b3']

In [19]:
uuids = [str(uuid4()) for _ in range(len(documents))]

In [20]:
vector_store.add_documents(documents=documents, ids=uuids)

['150928d3-3914-49fd-9518-7b6a53a802d2',
 '496b129e-f5c1-4317-a010-e2dd679098b7',
 '1a2898e1-c5a4-4587-945e-bdfd6c8cdec5',
 'd2971dbe-ef76-4323-be9d-a5cd4e0ce36a',
 'f3b856fb-96af-466c-bf4d-d756af185717',
 'f617b406-6afa-4da8-8177-927e15c25632',
 '59dee471-0f80-4985-a7e1-71ce341b1c19',
 '07aeacbd-fcaa-49ff-a1c5-f67032adc5fc',
 '131764cc-818a-44b4-9aa1-4eb8669ec653',
 'af3c4397-6f47-4f8c-917a-422e20016a06']

In [21]:
results = vector_store.similarity_search("what langchain provides to us?",k=1)

In [23]:
results

[Document(id='f28d1be9-5f8c-485a-bc0a-fd29982a48f2', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]